<a href="https://colab.research.google.com/github/hyeok1235/PyTorch/blob/main/Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisite Code
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:01<00:00, 18439086.14it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 344738.44it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6223257.18it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 11118577.24it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# Hyperparameters

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Loss Function

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Full Implementation

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  # Set the model to training mode - important for batch normalization and dropout layers
  # Unnecessary in this situation but added for best practices
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * batch_size + len(X)
      print(f"loss: {loss:>7f} [{current:>5d}.{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  # Set the model to evaluation mode - important for batch normalization and dropout layers
  # Unnecessary in this situation but added for best practices
  model.eval()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
  # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
loss_fn = nn.CrossEntropyLoss()
oprimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n----------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
----------------
loss: 0.703560 [   64.60000]
loss: 0.815594 [ 6464.60000]
loss: 0.572791 [12864.60000]
loss: 0.797523 [19264.60000]
loss: 0.691908 [25664.60000]
loss: 0.685236 [32064.60000]
loss: 0.761040 [38464.60000]
loss: 0.761460 [44864.60000]
loss: 0.732664 [51264.60000]
loss: 0.709487 [57664.60000]
Test Error: 
 Accuracy: 73.9%, Avg loss: 0.717408 

Epoch 2
----------------
loss: 0.674541 [   64.60000]
loss: 0.791438 [ 6464.60000]
loss: 0.549691 [12864.60000]
loss: 0.780050 [19264.60000]
loss: 0.675541 [25664.60000]
loss: 0.668300 [32064.60000]
loss: 0.740315 [38464.60000]
loss: 0.748778 [44864.60000]
loss: 0.715648 [51264.60000]
loss: 0.692387 [57664.60000]
Test Error: 
 Accuracy: 74.8%, Avg loss: 0.699678 

Epoch 3
----------------
loss: 0.648598 [   64.60000]
loss: 0.769117 [ 6464.60000]
loss: 0.529361 [12864.60000]
loss: 0.764508 [19264.60000]
loss: 0.660892 [25664.60000]
loss: 0.653740 [32064.60000]
loss: 0.721006 [38464.60000]
loss: 0.737233 [44864.60000]
loss: 0.7